### Algorithm overview

- Compute distance between each pair of in-nodes and out-nodes (geodesic distance on globe, later "distance" can be modified to include trade barriers, etc.)
- Probability/affinity of edge between (u,v) is proportional to in(u) * in(v) / dist(u,v)
- Connect nodes and assign weights according to iterative proportional fitting
- Validate data: pred_exp = sum(outflows from grid cells in i that are international edges) (same for imports)
- Note that the validation step doesn't actually test the network, it just looks at the net flow
- TODO: as of now, we just look at marginals (total imports and exports) rather than country flows. The model would be better if we train it on the marginals (force the network to match exisitng import and export values) and then compare it with actual data.

In [2]:
%load_ext autoreload
%autoreload 2

from FlowNetwork import FlowNetwork
import os

verbose = False

# Initialize the flow network with NetCDF data and variable names
grid_data_path = os.path.join('data', 'L.T.iron_flows.2000-2016.a.nc')
trade_csv_path = os.path.join('data', 'iron_raw_import_export.csv')

fn = FlowNetwork(grid_data_path, 'prod_outflow', 'prod_inflow', time='2000')
#fn.gravity_model(threshold_percentile=20, verbose=verbose)
#fn.ipf_flows(max_iters=100, tol=1e-6, verbose=verbose)
#fn.plot_network_map(caption=None, num_edges=500, vmin=None, vmax=None, color='coolwarm', levels=6, edge_thickness=1, edge_alpha=0.5, edge_color='gray')
#fn.check_mass_conservation(tol=1e-6, verbose=verbose)
#fn.analyze_degree_distribution(caption=None, verbose=verbose)
fn.compute_marginal_country_trade(out_path=None, verbose=verbose)
fn.compare_model_with_trade_data(trade_csv_path, "Fe_import", pred_col="estimated_imports", year=2000, raw_log=True, pred_log=True, rm_outliers=False)
fn.compare_model_with_trade_data(trade_csv_path, "Fe_export", pred_col="estimated_exports", year=2000, raw_log=True, pred_log=True, rm_outliers=False)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


UnboundLocalError: cannot access local variable 'tabular_stat' where it is not associated with a value